# 04_All_Experiments.ipynb — Failed Experiments & Ablations


## Configuration

In [7]:
import sys
import pandas as pd
import numpy as np
from scipy import sparse

# Ajout du path pour accéder au dossier src
sys.path.append('..')

from src.preprocessing import DataLoader
from src.metrics import mapk_score
# Import de toutes les classes expérimentales définies dans src/models/experimental.py
from src.models import (
    BM25Recommender,
    EASERecommender,
    SVDRecommender,
    DiversifiedRecommender,
    EnsembleHybridRecommender,
    HistoryFilterRecommender,
    LowInteractionRecommender,
    CoupledSemanticRecommender,
    SemanticHybridRecommenderChatGPT

)

# --- 1. CONFIGURATION & DONNÉES ---
print("--- Chargement des données ---")
loader = DataLoader('../data/interactions_train.csv', '../data/items.csv')
train_df, val_df = loader.get_time_split(train_ratio=0.8)

# Création de la Matrice de Validation (Ground Truth) une seule fois
print("Construction de la matrice de validation...")
val_rows = val_df['u_idx'].values
val_cols = val_df['i_idx'].values
val_data = np.ones(len(val_df))
val_matrix = sparse.csr_matrix(
    (val_data, (val_rows, val_cols)),
    shape=(loader.n_users, loader.n_items)
)

# Liste pour stocker les résultats
results = []

def run_experiment(model_class, name, **kwargs):
    """
    Fonction utilitaire pour entraîner, prédire et évaluer un modèle expérimental.
    """
    print(f"\n🧪 EXPÉRIENCE : {name}")
    try:
        # Instanciation
        model = model_class(loader.n_users, loader.n_items)

        # Entraînement
        print(f"   -> Entraînement avec params : {kwargs}...")
        model.fit(train_df, loader.items_df, **kwargs)

        # Prédiction
        print("   -> Prédiction...")
        preds = model.predict(k=10)

        # Évaluation
        score = mapk_score(preds, val_matrix, k=10)
        print(f"   -> ❌ Score MAP@10 : {score:.5f}")

        results.append({
            'Approche': name,
            'MAP@10': score,
            'Paramètres': str(kwargs),
            'Statut': 'Inférieur au Best Model'
        })

    except Exception as e:
        print(f"   -> 💥 CRASH : {e}")
        results.append({'Approche': name, 'MAP@10': 0.0, 'Paramètres': str(kwargs), 'Statut': 'Erreur/OOM'})

--- Chargement des données ---
Construction de la matrice de validation...


# --- 2. LANCEMENT DES EXPÉRIENCES ---

# ---------------------------------------------------------
# GROUPE A : Changement de Modèle Mathématique
# ---------------------------------------------------------

# 1. BM25 (Probabiliste)
# Idée : Mieux gérer la saturation des termes que TF-IDF.

In [8]:
run_experiment(BM25Recommender, "BM25 Probabilistic", k1=1.5, b=0.75)


🧪 EXPÉRIENCE : BM25 Probabilistic
   -> Entraînement avec params : {'k1': 1.5, 'b': 0.75}...
Fitting EXP: BM25 Model (k1=1.5, b=0.75)...
   -> Prédiction...
   -> ❌ Score MAP@10 : 0.19539


# 2. EASE (Linear Autoencoder)
# Idée : Apprendre la matrice de poids optimale directement.
# Problème : Très lourd en RAM, risque d'overfitting sur ce dataset.

In [9]:
run_experiment(EASERecommender, "EASE (Linear Model)", lambda_reg=500)


🧪 EXPÉRIENCE : EASE (Linear Model)
   -> Entraînement avec params : {'lambda_reg': 500}...
Fitting EXP: EASE Model (Lambda=500)...
 Computing Gram Matrix (Heavy RAM)...
   -> Prédiction...
   -> ❌ Score MAP@10 : 0.10668


# 3. SVD (Factorisation Matricielle)
# Idée : Capturer les facteurs latents cachés.
# Constat : Moins performant que le Content-Based sur ce dataset spécifique.

In [10]:
run_experiment(SVDRecommender, "SVD (Latent Factors)", n_factors=30)


🧪 EXPÉRIENCE : SVD (Latent Factors)
   -> Entraînement avec params : {'n_factors': 30}...
Fitting EXP: SVD Model (Factors=30)...
   -> Prédiction...
   -> ❌ Score MAP@10 : 0.04000


# ---------------------------------------------------------
# GROUPE B : Stratégies Temporelles Avancées
# ---------------------------------------------------------

# 4. Ensemble Short-Term / Long-Term
# Idée : Combiner un modèle 'récent' (7j) et un 'historique' (90j).
# Constat : Complexité inutile, un simple half-life de 30j suffit.

In [11]:
run_experiment(EnsembleHybridRecommender, "Ensemble Short/Long Term", half_lives=[7, 90])


🧪 EXPÉRIENCE : Ensemble Short/Long Term
   -> Entraînement avec params : {'half_lives': [7, 90]}...
Fitting EXP: Ensemble Model with Half-lives: [7, 90]...
   -> Sub-model 7 days...
   -> Sub-model 90 days...
   -> Prédiction...
   -> ❌ Score MAP@10 : 0.20263


# ---------------------------------------------------------
# GROUPE C : Filtrage et Contraintes
# ---------------------------------------------------------

# 5. Diversification (Auteur)
# Idée : Interdire plus de 2 livres du même auteur dans le top 10.
# Constat : Baisse mécanique du MAP@K car on retire des "vrais" items pertinents.

In [12]:
run_experiment(DiversifiedRecommender, "Diversification (Max 2/Author)")


🧪 EXPÉRIENCE : Diversification (Max 2/Author)
   -> Entraînement avec params : {}...
Fitting EXP: Diversified Model (Max 2 books/author)...
   -> Prédiction...
   -> ❌ Score MAP@10 : 0.19403


# 6. Filtrage Strict "Déjà Vu"
# Idée : Interdire de recommander un livre déjà lu (Score = -Infini).
# Constat : Baisse du score, le "re-buy" (ou relecture) est un signal positif ici.

In [13]:
run_experiment(HistoryFilterRecommender, "Filtre Strict Historique")


🧪 EXPÉRIENCE : Filtre Strict Historique
   -> Entraînement avec params : {}...
Fitting EXP: History Filter Model (No Re-buy allowed)...
   -> Prédiction...
   -> ❌ Score MAP@10 : 0.02075


# 7. Suppression Items Rares (Cold Start)
# Idée : Ignorer les items vus moins de 3 fois pour réduire le bruit.
# Constat : Perte d'information trop importante sur la "Long Tail".

In [14]:
run_experiment(LowInteractionRecommender, "Filtre Items Rares (<3 vues)", min_interactions=3)


🧪 EXPÉRIENCE : Filtre Items Rares (<3 vues)
   -> Entraînement avec params : {'min_interactions': 3}...
Fitting EXP: Low Interaction Filter (Min 3 views)...
   -> Dropped 6938 interactions on rare items.
   -> Prédiction...
   -> ❌ Score MAP@10 : 0.16306


# ---------------------------------------------------------
# GROUPE D : Architecture Sémantique (Itérations précédentes)
# ---------------------------------------------------------

# 8. Coupled Semantic Ensemble (Archived Version)
# Problème : Le boost de relecture était appliqué DANS la moyenne pondérée.
# Le modèle "court terme" (qui ne connait pas les vieux items) tirait la moyenne vers le bas pour les favoris.

In [15]:
run_experiment(
    CoupledSemanticRecommender,
    "Coupled Semantic (Re-buy Dilution)",
    alpha=0.5,
    half_life_days=[30, 150],
    ensemble_weights=[0.7, 0.3]
)


🧪 EXPÉRIENCE : Coupled Semantic (Re-buy Dilution)
   -> Entraînement avec params : {'alpha': 0.5, 'half_life_days': [30, 150], 'ensemble_weights': [0.7, 0.3]}...
Fitting EXP: Coupled Semantic Ensemble (Flawed Logic)...
   -> Prédiction...
   -> ❌ Score MAP@10 : 0.20452


 # ---------------------------------------------------------
 # GROUPE E : Enrichissement Sémantique (ChatGPT)
# ---------------------------------------------------------

# 9. Semantic Hybrid (ChatGPT Metadata),
 # Idée : Utiliser des descriptions et auteurs nettoyés par LLM pour améliorer le Content-Based

In [16]:
loader = DataLoader('../data/interactions_train.csv', '../data/items_enriched_ai_turbo.csv')

run_experiment(SemanticHybridRecommenderChatGPT, "Semantic Hybrid (ChatGPT Enriched)",
    alpha=0.5,
    half_life_days=[30, 150]
)


🧪 EXPÉRIENCE : Semantic Hybrid (ChatGPT Enriched)
   -> Entraînement avec params : {'alpha': 0.5, 'half_life_days': [30, 150]}...
Fitting SemanticHybrid Decoupled | Alpha=0.5, HL=[30, 150]...
Loading S-BERT & Encoding ENRICHED Metadata...


Batches: 100%|██████████| 473/473 [00:18<00:00, 25.72it/s]



--- Building Sub-Model 1 (Half-life=30d) ---

--- Building Sub-Model 2 (Half-life=150d) ---

Computing Global Popularity Scores...
Ensemble Model Fitted Successfully.
   -> Prédiction...
   -> ❌ Score MAP@10 : 0.20397


# --- 3. SYNTHÈSE DES RÉSULTATS ---

In [17]:
print("\n" + "="*50)
print("RÉSUMÉ FINAL DES ÉCHECS (FAILED EXPERIMENTS)")
print("="*50)

df_results = pd.DataFrame(results).sort_values('MAP@10', ascending=False)
pd.set_option('display.max_colwidth', None)
print(df_results[['Approche', 'MAP@10', 'Statut']])

print("\n--- ANALYSE DE L'ÉCHEC DU 'COUPLED SEMANTIC' ---")
print("L'expérience #8 montre que faire une moyenne pondérée naïve entre Court et Long terme")
print("est nuisible pour le 'Re-buy'. Le modèle court terme a 'oublié' les vieux favoris")
print("et dilue leur score lors de la fusion. La solution (Production) a été de séparer")
print("l'exploration (Ensemble) de l'exploitation (Re-buy sur historique global).")


RÉSUMÉ FINAL DES ÉCHECS (FAILED EXPERIMENTS)
                             Approche    MAP@10                   Statut
7  Coupled Semantic (Re-buy Dilution)  0.204524  Inférieur au Best Model
8  Semantic Hybrid (ChatGPT Enriched)  0.203973  Inférieur au Best Model
3            Ensemble Short/Long Term  0.202633  Inférieur au Best Model
0                  BM25 Probabilistic  0.195390  Inférieur au Best Model
4      Diversification (Max 2/Author)  0.194026  Inférieur au Best Model
6        Filtre Items Rares (<3 vues)  0.163061  Inférieur au Best Model
1                 EASE (Linear Model)  0.106677  Inférieur au Best Model
2                SVD (Latent Factors)  0.040002  Inférieur au Best Model
5            Filtre Strict Historique  0.020747  Inférieur au Best Model

--- ANALYSE DE L'ÉCHEC DU 'COUPLED SEMANTIC' ---
L'expérience #8 montre que faire une moyenne pondérée naïve entre Court et Long terme
est nuisible pour le 'Re-buy'. Le modèle court terme a 'oublié' les vieux favoris
et dil